In [1]:
### basic package for data science project
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
sns.set()

In [2]:
train_data = pd.read_csv('data/exercise_40_train.csv')

In [ ]:
most_missing_cols = set(df.columns[df.isnull().mean() > 0.4])

In [ ]:
def clean_train_data(df):
    

In [ ]:
def extract_features(df):